# Bootcamp Data Science Aplicada - Alura

# Módulo 02 - Visualização de Dados com Seaborn e Matplotlib

**Obs.** Notabook para o acompanhamento das aulas e resolução dos desafios propostos. Caso necessário, dividirei em mais de um notebook.

## Aula 01 - Fontes Externas, Limpeza e Manipulação de Dados

Iniciou-se revisando o notebook do módulo anterior e comparando o valor "bruto" com o valor absoluto de gastos.

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from io import StringIO

# bibliotecas utilizadas na parte de web scraping
import requests
from bs4 import BeautifulSoup

# fomatação dos números para duas casas decimais
pd.options.display.float_format = "{:.2f}".format

### Resumo dos dataframes usados no módulo passado, para caso algum seja reutilizado nesse módulo 2

In [41]:
dados = pd.read_csv("https://raw.githubusercontent.com/alura-cursos/agendamento-hospitalar/main/dados/A160324189_28_143_208.csv",
                    encoding = 'ISO-8859-1', skiprows = 3, sep = ';',
                    skipfooter = 12, thousands = '.', decimal = ',', engine = 'python')

In [42]:
colunas_usaveis = dados.mean().index.tolist()
colunas_usaveis.insert(0, "Unidade da Federação")
usaveis = dados[colunas_usaveis]
usaveis = usaveis.set_index("Unidade da Federação")
usaveis = usaveis.drop("Total", axis = 1)
usaveis["Total"] = usaveis.sum(axis = 1)
ordenado_por_total = usaveis.sort_values("Total", ascending = False)
ordenado_por_total = ordenado_por_total.drop("Total", axis = 1)
ordenado_por_total = ordenado_por_total / 1000000
meses_filtrados = ordenado_por_total.columns[47:150]
mes_mais_recente = ordenado_por_total.columns[-1]
gastos_do_mais_recente = ordenado_por_total[mes_mais_recente]
gastos_do_mais_recente = gastos_do_mais_recente.sort_values(ascending = False)
gastos_proporcionais = gastos_do_mais_recente / gastos_do_mais_recente.sum()

### Aulas 01 & 02

In [43]:
ibge_estimativa = pd.read_excel("https://github.com/alura-cursos/agendamento-hospitalar/raw/main/dados/estimativa_dou_2020.xls")
ibge_estimativa.head()

,ESTIMATIVAS DA POPULAÇÃO RESIDENTE NO BRASIL E UNIDADES DA FEDERAÇÃO COM DATA DE REFERÊNCIA EM 1º DE JULHO DE 2020,Unnamed: 1,Unnamed: 2
0,BRASIL E UNIDADES DA FEDERAÇÃO,NaN,POPULAÇÃO ESTIMADA
1,Brasil,NaN,211755692
2,Região Norte,NaN,18672591
3,Rondônia,NaN,1796460
4,Acre,NaN,894470


In [44]:
dados_da_populacao = """Posição	Unidade federativa	População	% da pop. total	País comparável
(habitantes)

1	 São Paulo	46 289 333	21,9%	Flag of Spain.svg Espanha (46 439 864)
2	 Minas Gerais	21 292 666	10,1%	 Sri Lanka (20 675 000)
3	 Rio de Janeiro	17 366 189	8,2%	 Países Baixos (16 922 900)
4	Bahia Bahia	14 930 634	7,1%	 Chade (14 037 000)
5	 Paraná	11 516 840	5,4%	 Bolívia (11 410 651)
6	 Rio Grande do Sul	11 422 973	5,4%	 Bélgica (11 250 659)
7	 Pernambuco	9 616 621	4,5%	 Bielorrússia (9 485 300)
8	 Ceará	9 187 103	4,3%	 Emirados Árabes Unidos (9 157 000)
9	Pará Pará	8 690 745	4,1%	 Áustria (8 602 112)
10	 Santa Catarina	7 252 502	3,4%	 Sérvia (7 114 393)
11	 Maranhão	7 114 598	3,4%	 Paraguai (7 003 406)
12	 Goiás	7 113 540	3,4%	 Paraguai (7 003 406)
13	 Amazonas	4 207 714	2,0%	 Líbano (4 168 000)
14	 Espírito Santo	4 064 052	1,9%	 Líbano (4 168 000)
15	 Paraíba	4 039 277	1,9%	 Líbano (4 168 000)
16	 Rio Grande do Norte	3 534 165	1,7%	 Uruguai (3 415 866)
17	 Mato Grosso	3 526 220	1,7%	 Uruguai (3 415 866)
18	 Alagoas	3 351 543	1,6%	 Uruguai (3 415 866)
19	 Piauí	3 281 480	1,6%	 Kuwait (3 268 431)
20	 Distrito Federal	3 055 149	1,4%	 Lituânia (2 900 787)
21	 Mato Grosso do Sul	2 809 394	1,3%	 Jamaica (2 717 991)
22	 Sergipe	2 318 822	1,1%	 Namíbia (2 280 700)
23	 Rondônia	1 796 460	0,8%	 Gabão (1 725 000)
24	 Tocantins	1 590 248	0,7%	 Bahrein (1 359 800)
25	 Acre	894 470	0,4%	 Fiji (859 178)
26	 Amapá	861 773	0,4%	 Fiji (859 178)
27	 Roraima	631 181	0,3%	 Luxemburgo (562 958)"""

# fonte: https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o

In [47]:
# converte a string para stringIO
dados_da_populacao_io = StringIO(dados_da_populacao)

# lê os dados que colamos acima em um pandas dataframe
populacao = pd.read_csv(dados_da_populacao_io, sep="\t")

In [48]:
populacao.head()

,Posição,Unidade federativa,População,% da pop. total,País comparável
0,(habitantes),NaN,NaN,NaN,NaN
1,1,São Paulo,46 289 333,"21,9%",Flag of Spain.svg Espanha (46 439 864)
2,2,Minas Gerais,21 292 666,"10,1%",Sri Lanka (20 675 000)
3,3,Rio de Janeiro,17 366 189,"8,2%",Países Baixos (16 922 900)
4,4,Bahia Bahia,14 930 634,"7,1%",Chade (14 037 000)


- Um referência interessante sobre **StringIO**: [Manipulando Strings Como Se Fossem Arquivos](https://pythonhelp.wordpress.com/2014/06/19/manipulando-strings-como-se-fossem-arquivos-stringio/)

## Desafio 01 - Entre nos dados da Wikipédia e leia a base de dados direto da Wikipédia.

Para resolver esse desafio precisamos usar da técnica de **web scraping**.

- Link para obtenção dos dados: https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o

- [Web Scraping a Wikipedia Table into a Dataframe](https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451)

In [50]:
url_wiki = 'https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o'

Há muitas controvérsias sobre a legalidade de se realizar web scraping, mas, no nosso caso, a Wikipédia permite realizarmos o web scraping para fins de estudo, e de uma forma responsável, sem mandar muitos _requests_ ao mesmo tempo para a página.

Então vamos em frente.